# Let's Build Personalized Website

Notes: 
1. Use the native Jupyter Notebook to leverage the full features of this notebook
2. Ensure you have Bedrock connection with access to Titan Models 

## 1. Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import base64
import boto3
import io
import os
import json
import re
import importlib
import shutil
import numpy as np
import matplotlib.pyplot as plt

from src import Bedrock

from ipywidgets import widgets
from IPython.core.display import display, HTML
from PIL import Image


In [ ]:
importlib.reload(Bedrock)

### Useful functions

In [ ]:
# fetch context from the response
def get_contexts(retrievalResults):
    total_text = ""
    for result in retrievalResults.get("retrievalResults", []):
        content = result.get("content", {})
        text = content.get("text", "")
        # Remove the metadataAttributes content from the text
        text = re.sub(r'\{.*?metadataAttributes.*?\}', '', text)
        total_text += text
    return total_text

In [ ]:
def build_profile(UserProfile):
    profile = 'Your customer is {}. Their industry is {}. They have {} employees. They are a {} company, located in {}.' \
    ' Their mission statement is `{}`.'.format(UserProfile['Name'],
                                UserProfile['Industry'],
                                UserProfile['CompanySize'],
                                UserProfile['CompanyType'],
                                UserProfile['Location'],
                                UserProfile['Mission'])
    return profile

### Decelerations

In [ ]:
# create general model inputs 
# do not edit any content in this section

image_modelId = 'amazon.titan-image-generator-v1'
model_id = {'Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
           'Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
           'StableDiff': 'stability.stable-diffusion-xl-v1'}

In [ ]:
# ENV VARIABLES
kbId = '9OIYJNO1EE'

### INPUT

Offerings

In [ ]:
with open("./references/offerings.json", "r") as file:
    offerings = json.load(file)

User Profiles

In [ ]:
Profiles = {
    'Construction-Example': {
        'Name': 'Example Corp Construction',
        'Industry': 'Construction',
        'CompanySize': 1500,
        'CompanyType': 'Enterprise',
        'Location': 'New York City, NY',
        'Mission': 'Building a sustainable future for New York'
    },
    'Manufacturing-Example': {
        'Name': 'Example Corp Manuf',
        'Industry': 'Manufacturing',
        'CompanySize': 2500,
        'CompanyType': 'Enterprise',
        'Location': 'San Jose, CA',
        'Mission': 'Building the next generation Electric Vehicles'
    },
    'Mining-Example': {
        'Name': 'Example Corp Mining',
        'Industry': 'Mining',
        'CompanySize': 500,
        'CompanyType': 'Midsize',
        'Location': 'Bisbee, AZ',
        'Mission': 'Extracting the value for America'
    }
}

Now a user signs in (ITERATION)

In [ ]:
# ITERATION - SELECTION:
selected_profile = 'Manufacturing-Example'

In [ ]:
UserProfile= Profiles[selected_profile]
offering = offerings[UserProfile['Industry']]
project = selected_profile + "_Website_Haiku"

### Runtime

#### 1) Natural Language description of customer profile

In [ ]:
customer = build_profile(UserProfile)
print(customer)

#### 2) Retreieve painpoints of customer industry / any context
This flow uses RAG to retreive documents

In [ ]:
query = '{}\nWhat are the possible pain-points and challanges for this customer?'.format(customer)
print(query)

In [ ]:
context_painpoints = Bedrock.dec_retrieve(query, kbId, numberOfResults=10)
contexts_painpoints = get_contexts(context_painpoints)
print(contexts_painpoints)

### 4) Prompting AI to describe a website personalized for this profile and background
Here the prompting is critical. Please see the guidance provided to the LLM. We also ask the response to be in two parts; 1/ Description, 2/ Visual Elements. The second part is needed for an easier traffic move to image generation or finding image.

In [ ]:
testimonials = open("./references/Testimonials.txt", "r").read()
print(testimonials)

First, we need to get the most recent suggested steps for the LLM

In [ ]:
design_steps= open("./references/RecommendedStepsToDesignNew.txt", "r").read()
print(design_steps)

In [ ]:
design_guideline= open("./references/DesignGuidelineNew.txt", "r").read()

Now, we have all the inputs to form our prompt

In [ ]:
prompt = f"""
You are a passionate AI UI/UX designer working for AnyCompany Consulting, a consulting firm that provides management and technical consultancy services to companies in the construction, mining, and manufacturing industries. AnyCompany Consulting now offers personalized online experiences for their registered customers by creating tailored landing pages. These websites feature offerings, design elements, and calls to action that are customized to each customer's profile and industry-specific pain points.

Your task is to design visually appealing and personalized landing pages personalized for customers to encourage them to work with AnyCompany Consulting to get consultancy services. Your response in this activity will be used by artists and front-end developers.

The personalized website should be composed of the following sections:
- A Header
- A Hero Section
- An Offerings Section
- A Customer Pain-points Section
- A Testimonials section 
- A Call-to-Action Section

The website should display the company logo (anycompany_logo.jpg) and the website title is "AnyCompany Consulting Personalized Page". Please also consider adding common menu options on the top.  

The customer testimonials are:
{testimonials}

Please follow the below design guidelines:

{design_guideline}

Your task is to design AnyCompany Consulting landing page website personalized for the below customer:

Customer Profile:
{customer}

Related offerings:
<offerings>
{offering}
</offerings>

Customer challenges and pain-points:
<pain-points>
{contexts_painpoints}
</pain-points>

First, design the personalized website following these steps:

{design_steps}

Now, please provide your response in two sections:

Section 1: Detailed Website Description
This section will be used by front-end developers. Provide a comprehensive and complete description of the personalized website, dynamic content for JavaScript, including the layout, images, sections, and content with full text. Be specific and elaborate in your descriptions. Use labels (e.g. hero-example.jpg) when mentioning images.

Section 2: Visual Elements
This section will be used by artists and designers. Provide a consolidated list of all images with their labels and detailed descriptions. Tag each item with <VISUAL_LABEL> for the label and <VISUAL_DESCRIPTION> for the description, following the provided example format.

Example:

1. <VISUAL_LABEL>hero-image.jpg</VISUAL_LABEL>
<VISUAL_DESCRIPTION>This image depicts a bustling construction site with workers actively engaged in various tasks. In the foreground, there are workers operating heavy machinery, such as excavators and cranes, while others are seen working on scaffolding or carrying materials. The background features a partially constructed building, showcasing the progress of the project.</VISUAL_DESCRIPTION>

2. <VISUAL_LABEL>service-icon.png</VISUAL_LABEL>
<VISUAL_DESCRIPTION>This icon represents the "Pre-Construction Services". It features a blueprint or architectural drawing as the central element, symbolizing the planning and preparation phase of construction projects.</VISUAL_DESCRIPTION>

3. ...
4. ...
"""

In [ ]:
%%time
response_personalized_website = Bedrock.invoke(prompt=prompt, modelID=model_id['Haiku'], max_tokens = 4096, temp = 0.0)

In [ ]:
print(response_personalized_website)

#### 5) Image Creation based on Description


##### a) Get the Picture information from the response of the Personalizer LLM
Notice that if the image is "image" then we use 1024 x 1024 otherwise 512 x 512. 

In [ ]:
pattern = r'<VISUAL_LABEL>(.+?)</VISUAL_LABEL>\s*<VISUAL_DESCRIPTION>(.+?)</VISUAL_DESCRIPTION>'

# Find all matches in the input string
matches = re.findall(pattern, response_personalized_website, re.DOTALL)


# Create a list of tuples with visual asset name and description
visual_assets= {m[0].strip(): m[1].strip() for m in matches}

# Creating visual assets with the right sizing
visuals = []
for asset in visual_assets.items():
    if "anycompany_logo" in asset[0]:
        continue  # Skip this iteration if the filename contains "unicorn"
    else:
        width = 1024
        height = 1024
    visuals.append({'FileName': asset[0], 'Prompt': asset[1], 'width':width, 'height':height})

In [ ]:
print("Number of visual assets created: {}".format(len(visuals)))

##### b) Invoke Stable Diffusion
Here we assumed that the image description is the prompt for Text-to-Image. We successively call the APIs and then save the files.

In [ ]:
%%time
responses_image = []
images = []
for vis in visuals:
    response = Bedrock.invoke_stable_diff(prompt = vis['Prompt'],  seed=0, height = vis['height'], 
                                          width = vis['width'], steps = 30, cfg_scale = 10)
    responses_image.append(response)
    image_bytes = base64.b64decode(response.encode('ascii'))

    # Save the image
    image = Image.open(io.BytesIO(image_bytes))
    image.save(f"{vis['FileName']}")
    images.append(np.array(image))

We also copy the company logo from references to next to other visuals for better visibility

In [ ]:
shutil.copy("./references/anycompany_logo.jpg", "./anycompany_logo.jpg")

Let's plot the images generated

In [ ]:
# Plot the images
num_images = len(images)
fig, axs = plt.subplots(1, num_images, figsize=(20, 10))

for i, (image, ax, filename) in enumerate(zip(images, axs, [vis['FileName'] for vis in visuals])):
    ax.imshow(image)
    ax.set_title(filename, fontsize=16)
    ax.axis('off')

plt.tight_layout()
plt.show()

#### 6) Let's Generate our HTML file

First, we need to extract the response for the Section 1: Detailed Description from *response_personalized_website*

In [ ]:
# Extract Section-1 using regular expressions
#section_1_pattern = r"Section 1: Detailed Website Description\n\n(.*?)\n\nSection 2:"
#match = re.search(section_1_pattern, response_personalized_website, re.DOTALL)

#if match:
#    response_detailed_explanation = match.group(1)
#    print(f"Here is the Section-1 extracted from the LLM response:\n\n{response_detailed_explanation}")
#else:
#    print("Section-1 not found in the input text.")

Here we will use Haiku to create the HTML file.

In [ ]:
prompt = f"""
You are an experienced front-end web developer specializing in creating accessible, responsive, and visually appealing websites. Your task is to generate the complete HTML, CSS, and JavaScript code that accurately implements the provided 'Website Description' while adhering to the specified guidelines.

<website description>
Know that, this your Design Guideline (Requirements):
{design_guideline}

You use the testimonials as follows;
{testimonials}

Website Description:
{response_personalized_website}
</website description>

Please carefully read the 'Website Description' line by line, and then generate the HTML, CSS, and JavaScript code required to build the described website while following the specified design guidelines and requirements.

Provide the HTML, CSS, and JavaScript code directly, starting with the <!DOCTYPE html> declaration, without any preamble or introduction.

"""

In [ ]:
%%time
response_html = Bedrock.invoke(prompt=prompt, modelID=model_id['Haiku'], max_tokens = 4096, temp = 0)

In [ ]:
#print(response_html)

#### Putting the assets into a single folder

In [ ]:
#Creating the folder for our project
try:
    os.mkdir(project)
    print(f"Project Folder: {project}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from IPython.display import HTML

In [ ]:
with open('main.html', 'w', encoding='utf-8') as file:
    file.write(response_html)

In [ ]:
project_files = list(visual_assets.keys())
project_files.append('main.html')

In [ ]:
for file in project_files:
    destination = project + "/" + file
    shutil.move(file, destination)